<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=8d98f0774cf57ef5969076881df438d27922b9913d10515b1d2604978c3e3ee4
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.4
    Uninstalling protobuf-6.33.4:
      Successfully uninstalled protobuf-6.33.4


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-13 09:27:55
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.41 C
-------------------
Today PnL: 35.59 K (0.25%)
Current PnL: -27.97 L (-17.84%)
CY Booked + Current PnL: -13.68 L (-8.72%)
-------------------
Total profit:  1.75 L
Total loss:  -29.72 L
-------------------
Total Booked + Current PnL: 13.30 L (10.25%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.13%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 93.42 L (66.21%)
Deployed:  1.30 C
Current:  1.41 C
CAGR/XIRR %: 4.51%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,0.12,15.84,4.79,21.40,14728.0,42050.0,307476.0,7.47,64.0,X-LC,6.12,14.0,2.86,2.18,23.17,XY25,NTT,FMCG
17,COALINDIA,484.83,-0.21,6.09,12.39,19.23,19777.0,9158.0,159618.0,24.27,71.0,L-LC,8.34,182.0,0.46,1.13,28.66,XY25,ATH,MINING
77,TTKPRESTIG,770.00,0.23,-22.15,28.57,0.09,22415.0,-22321.0,78456.0,82.07,35.0,M-SC,3.46,253.0,-1.00,0.56,3.07,OX40N,NTT,DURABLES
39,INDIGOPNTS,1408.00,0.78,-14.27,16.67,0.02,24945.0,-24916.0,149643.0,128.24,54.0,M-SC,8.59,240.0,-1.00,1.06,30.16,OX40N,NTT,PAINTS
49,MASFIN,398.61,-0.03,-3.97,27.09,22.05,25490.0,-3885.0,94095.0,-17.34,47.0,H-SC,5.72,168.0,-0.15,0.67,36.77,XR,ATH,FINANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
5,ASIANPAINT,3460.25,2.51,1.24,19.47,20.95,49626.0,3115.0,254883.0,0.21,63.0,X-LC,12.42,36.0,0.06,1.81,37.31,X40,ATH,PAINTS
26,GILLETTE,11206.78,1.93,-5.56,40.21,32.42,96420.0,-14106.0,239790.0,-22.30,43.0,X-SC,12.34,87.0,-0.15,1.70,8.70,X40,ATH,FMCG
80,VALIANTORG,838.00,1.72,-45.46,219.30,74.15,195688.0,-74372.0,89233.0,-260.32,44.0,H-SC,21.58,145.0,-0.38,0.63,14.38,XR,NTT,CHEMICALS
46,JSWINFRA,342.00,1.67,-8.56,27.33,16.43,55423.0,-18981.0,202793.0,-22.54,40.0,X-MC,7.34,66.0,-0.34,1.44,19.68,X40N,NTT,REALTY
55,RAJESHEXPO,518.00,1.65,-67.93,212.05,0.07,93281.0,-93187.0,43990.0,1512.37,35.0,L-SC,1.46,269.0,-1.00,0.31,9.40,OX40N,NTT,JEWELLERY


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,1972.00,-1.13,8.64,23.57,34.24,80121.0,27039.0,339927.0,-15.29,46.0,X-LC,7.63,6.0,0.34,2.41,17.64,X40,NTT,IT
59,RELIANCE,1952.00,-0.90,-5.65,32.80,25.30,62194.0,-11346.0,189617.0,-8.81,34.0,X-LC,7.43,26.0,-0.18,1.34,27.01,XY25,BTT,REFINERIES
32,HCLTECH,1922.01,-0.70,7.47,16.06,24.73,41755.0,18068.0,259992.0,12.34,54.0,X-LC,6.14,13.0,0.43,1.84,23.84,X40,ATH,IT
44,JCHAC,2282.00,-0.67,-35.68,55.46,-0.01,45589.0,-45603.0,82202.0,16726.09,45.0,M-SC,13.35,235.0,-1.00,0.58,5.21,OX40N,BTT,AC
29,HAPPSTMNDS,1480.71,-0.60,-46.25,243.91,84.85,168025.0,-59277.0,68888.0,-36.02,29.0,H-SC,16.16,153.0,-0.35,0.49,0.00,AR,ATH,IT


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,-0.09,0.2,20.49,20.74,41714.0,412.0,203582.0,31.15,56.0,M-MC,5.46,189.0,0.01,1.44,10.84,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.79,-3.41,115.24,107.91,164351.0,-5028.0,142616.0,-21.36,48.0,M-SC,10.16,220.0,-0.03,1.01,6.34,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,0.78,-14.27,16.67,0.02,24945.0,-24916.0,149643.0,128.24,54.0,M-SC,8.59,240.0,-1.00,1.06,30.16,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,0.84,-17.25,59.56,32.03,64973.0,-22747.0,109088.0,-10.50,66.0,H-SC,6.71,157.0,-0.35,0.77,22.19,OX40N,NTT,IT
77,TTKPRESTIG,770.00,0.23,-22.15,28.57,0.09,22415.0,-22321.0,78456.0,82.07,35.0,M-SC,3.46,253.0,-1.00,0.56,3.07,OX40N,NTT,DURABLES
47,KANSAINER,340.00,0.55,-22.60,46.61,13.47,97280.0,-60957.0,208710.0,-68.12,47.0,H-SC,4.37,159.0,-0.63,1.48,6.12,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,-0.09,0.20,20.49,20.74,41714.0,412.0,203582.0,31.15,56.0,M-MC,5.46,189.0,0.01,1.44,10.84,XY25,NTT,FINANCE
17,COALINDIA,484.83,-0.21,6.09,12.39,19.23,19777.0,9158.0,159618.0,24.27,71.0,L-LC,8.34,182.0,0.46,1.13,28.66,XY25,ATH,MINING
84,WIPRO,420.00,0.08,7.99,59.51,72.27,108900.0,13546.0,182994.0,-7.44,51.0,M-LC,6.27,101.0,0.12,1.30,14.66,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.08,7.99,59.51,72.27,108900.0,13546.0,182994.0,-7.44,51.0,M-LC,6.27,101.0,0.12,1.30,14.66,XR,NTT,IT
25,FINCABLES,1641.55,0.79,-3.41,115.24,107.91,164351.0,-5028.0,142616.0,-21.36,48.0,M-SC,10.16,220.0,-0.03,1.01,6.34,OX40N,ATH,CABLES
38,INDIAMART,4810.62,-0.09,-7.53,123.55,106.72,140910.0,-9285.0,114051.0,-54.25,36.0,H-SC,7.82,138.0,-0.07,0.81,14.09,AR,ATH,MISC
37,IEX,219.00,0.01,-3.61,54.19,48.63,104443.0,-7219.0,192735.0,-35.77,51.0,H-SC,13.16,137.0,-0.07,1.37,7.59,XR,NTT,MISC
85,ZYDUSLIFE,1286.17,-0.00,-5.62,43.70,35.63,86441.0,-11768.0,197806.0,-18.95,36.0,H-MC,3.44,119.0,-0.14,1.40,10.12,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,0.13,-16.95,33.39,10.78,66641.0,-40735.0,199583.0,-50.55,15.0,X-LC,7.86,1.0,-0.61,1.41,0.50,X40,NTT,FMCG
7,AWL,485.00,1.22,-27.89,117.98,57.19,277468.0,-90945.0,235182.0,-65.93,27.0,X-MC,9.94,58.0,-0.33,1.67,1.22,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-0.20,-41.43,130.84,35.20,98606.0,-53306.0,75364.0,-1.23,27.0,X-SC,17.51,93.0,-0.54,0.53,0.00,X40,NTT,FOOTWEAR
52,PAGEIND,51605.08,0.22,-12.83,49.84,30.62,102989.0,-30414.0,206640.0,-38.08,28.0,X-MC,16.85,57.0,-0.30,1.46,0.47,X40,ATH,APPARELS
78,UNITDSPR,1644.00,-0.07,-2.54,23.92,20.77,54584.0,-5952.0,228192.0,-11.08,30.0,X-MC,1.57,70.0,-0.11,1.62,4.20,X40N,NTT,BREWERIES


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.00,-0.07,-2.54,23.92,20.77,54584.0,-5952.0,228192.0,-11.08,30.0,X-MC,1.57,70.0,-0.11,1.62,4.20,X40N,NTT,BREWERIES
45,JIOFIN,387.00,0.14,-7.15,34.07,24.48,84575.0,-19126.0,248239.0,-8.72,40.0,X-LC,1.59,37.0,-0.23,1.76,43.89,XY24,BTT,FINANCE
65,SIEMENS,4671.50,0.77,-19.29,55.51,25.51,83373.0,-35900.0,150195.0,-7.73,40.0,H-LC,2.12,50.0,-0.43,1.06,10.68,AR,ATH,ELECTRICAL
20,DABUR,735.00,0.40,2.86,39.71,43.70,101323.0,7090.0,255158.0,-5.58,69.0,X-MC,2.18,73.0,0.07,1.81,18.34,XY24,BTT,FMCG
73,TCS,4389.97,-0.56,-11.43,36.28,20.71,121543.0,-43222.0,335015.0,-23.62,49.0,X-LC,2.24,3.0,-0.36,2.37,11.94,X40,ATH,IT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.20,-41.43,130.84,35.20,98606.0,-53306.0,75364.0,-1.23,27.0,X-SC,17.51,93.0,-0.54,0.53,0.00,X40,NTT,FOOTWEAR
58,RELAXO,1176.00,-0.05,-49.02,198.78,52.31,147091.0,-71163.0,73997.0,-45.70,35.0,X-SC,7.85,92.0,-0.48,0.52,0.00,X40N,NTT,FOOTWEAR
8,BAJAJHFL,181.50,0.33,-19.65,93.50,55.47,182773.0,-47807.0,195479.0,-28.89,35.0,X-MC,15.77,63.0,-0.26,1.39,0.33,X40N,ATH,FINANCE
52,PAGEIND,51605.08,0.22,-12.83,49.84,30.62,102989.0,-30414.0,206640.0,-38.08,28.0,X-MC,16.85,57.0,-0.30,1.46,0.47,X40,ATH,APPARELS
43,ITC,452.00,0.13,-16.95,33.39,10.78,66641.0,-40735.0,199583.0,-50.55,15.0,X-LC,7.86,1.0,-0.61,1.41,0.50,X40,NTT,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,0.07,-28.58,100.02,42.84,46433.0,-18582.0,46424.0,-53.17,48.0,X-SC,25.97,83.0,-0.40,0.33,5.03,XY24,NTT,MISC
58,RELAXO,1176.00,-0.05,-49.02,198.78,52.31,147091.0,-71163.0,73997.0,-45.70,35.0,X-SC,7.85,92.0,-0.48,0.52,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.20,-41.43,130.84,35.20,98606.0,-53306.0,75364.0,-1.23,27.0,X-SC,17.51,93.0,-0.54,0.53,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,1.57,-0.32,24.48,24.08,30978.0,-407.0,126543.0,-8.38,50.0,X-SC,6.13,91.0,-0.01,0.90,19.23,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,0.53,-18.12,75.04,43.32,100073.0,-29512.0,133360.0,-29.66,42.0,X-SC,8.08,90.0,-0.29,0.95,2.40,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,0.13,-16.95,33.39,10.78,66641.0,-40735.0,199583.0,-50.55,15.0,X-LC,7.86,1.0,-0.61,1.41,0.50,X40,NTT,FMCG
75,TMPV,600.00,-0.47,-28.49,71.97,22.98,116261.0,-64357.0,161541.0,-25.52,37.0,X-LC,3.82,2.0,-0.55,1.14,1.60,XY24,NTT,AUTO
73,TCS,4389.97,-0.56,-11.43,36.28,20.71,121543.0,-43222.0,335015.0,-23.62,49.0,X-LC,2.24,3.0,-0.36,2.37,11.94,X40,ATH,IT
81,VBL,671.64,-0.17,-0.42,36.25,35.67,114007.0,-1340.0,314502.0,-12.33,55.0,X-LC,2.36,5.0,-0.01,2.23,13.37,X40N,ATH,FMCG
41,INFY,1972.00,-1.13,8.64,23.57,34.24,80121.0,27039.0,339927.0,-15.29,46.0,X-LC,7.63,6.0,0.34,2.41,17.64,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,-0.57,-33.53,99.93,32.89,51281.0,-25890.0,51317.0,-733.49,62.0,L-MC,8.13,259.0,-0.50,0.36,41.32,XR,NTT,BANKS
13,BSOFT,831.70,0.58,-24.63,100.05,50.79,106069.0,-34637.0,106016.0,-3.36,45.0,H-SC,10.21,171.0,-0.33,0.75,23.44,XR,ATH,IT
69,SURYODAY,216.00,-0.34,-18.77,51.47,23.03,74864.0,-33619.0,145452.0,57.89,53.0,H-SC,4.13,165.0,-0.45,1.03,44.32,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,0.78,-14.27,16.67,0.02,24945.0,-24916.0,149643.0,128.24,54.0,M-SC,8.59,240.0,-1.00,1.06,30.16,OX40N,NTT,PAINTS
17,COALINDIA,484.83,-0.21,6.09,12.39,19.23,19777.0,9158.0,159618.0,24.27,71.0,L-LC,8.34,182.0,0.46,1.13,28.66,XY25,ATH,MINING


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,0.24,-29.55,60.27,12.90,92768.0,-64576.0,153920.0,127.16,60.0,H-SC,9.66,173.0,-0.70,1.09,27.98,OX40N,NTT,FINANCE
72,TATAELXSI,9161.00,0.84,-17.25,59.56,32.03,64973.0,-22747.0,109088.0,-10.50,66.0,H-SC,6.71,157.0,-0.35,0.77,22.19,OX40N,NTT,IT
17,COALINDIA,484.83,-0.21,6.09,12.39,19.23,19777.0,9158.0,159618.0,24.27,71.0,L-LC,8.34,182.0,0.46,1.13,28.66,XY25,ATH,MINING
57,RECLTD,446.00,-0.09,0.20,20.49,20.74,41714.0,412.0,203582.0,31.15,56.0,M-MC,5.46,189.0,0.01,1.44,10.84,XY25,NTT,FINANCE
20,DABUR,735.00,0.40,2.86,39.71,43.70,101323.0,7090.0,255158.0,-5.58,69.0,X-MC,2.18,73.0,0.07,1.81,18.34,XY24,BTT,FMCG


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.35
1,25,44.43
2,50,76.57


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.58
MC    30.78
LC    26.63
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.29
X40      23.97
X40N     12.80
XY25     10.85
XR        8.98
AR        8.53
OX40N     7.63
SR        0.94
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.40
X-LC    23.14
H-SC    22.78
M-SC    11.22
X-SC     7.80
H-MC     4.58
M-MC     1.44
M-LC     1.30
L-LC     1.13
H-LC     1.06
L-SC     0.78
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.26,-8.13,43.61
IT,12.85,-17.93,77.41
FINANCE,11.05,-14.41,62.43
MISC,7.02,-31.12,84.75
ELECTRICAL,5.87,-13.33,54.84
PAINTS,5.82,-12.58,29.30
INSURANCE,4.72,-1.88,36.75
PHARMA,3.97,-4.87,37.89
AUTO,2.77,-34.01,80.47


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3263334.0,21
AR,1367951.0,10
XR,1225573.0,12
X40,1136557.0,15
X40N,937903.0,9
OX40N,713265.0,10
XY25,410025.0,7
SR,287796.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3580918.0,24
X-MC,1593656.0,16
M-SC,1476095.0,15
X-LC,955760.0,13
X-SC,818268.0,8
H-MC,431403.0,3
L-SC,181259.0,2
M-LC,108900.0,1
H-LC,83373.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1266038.0      6
           AR         927105.0      5
M-SC       XY24       846679.0      6
H-SC       XR         789695.0      7
X-MC       X40        520115.0      7
           XY24       447231.0      3
X-LC       X40        421416.0      6
X-MC       X40N       405849.0      4
X-SC       X40N       326619.0      3
H-SC       OX40N      310284.0      4
M-SC       OX40N      309700.0      5
X-SC       XY24       296623.0      3
H-SC       SR         287796.0      2
H-MC       AR         225476.0      2
X-MC       XY25       220461.0      2
H-MC       XY24       205927.0      1
X-LC       X40N       205435.0      2
           XY24       200836.0      2
X-SC       X40        195026.0      2
M-SC       XR         187719.0      2
           AR         131997.0      2
X-LC       XY25       128073.0      3
M-LC       XR         108900.0      1
L-SC       OX40N       93281.0      1
           XR          87978.0      1
H-LC       AR          83373.0      1
L-MC       XR          51281.0      1
M-MC       XY25        41714.0      1
L-LC       XY25        19777.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 34.0 seconds
